In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV


In [39]:
# Read data sets
X_train = pd.read_csv('ais_train.csv', sep='|')
X_test = pd.read_csv('ais_test.csv')

# Import ports data
ports = pd.read_csv("ports.csv", sep="|")

# Import vessels data
vessels = pd.read_csv("vessels.csv", sep="|")

#Import schedules data
schedules = pd.read_csv("schedules_to_may_2024.csv", sep="|")

## Preprocessing and feature engineering

In [3]:
def preprocess(df_train,df_test,df_schedules,df_ports,df_vessels):  
    
    train = df_train.copy()
    test = df_test.copy()
    schedules = df_schedules.copy()
    ports = df_ports.copy()
    vessels = df_vessels.copy()

    # Format time
    train['time'] = pd.to_datetime(train['time'])
    test['time'] = pd.to_datetime(test['time'])

    # Factorize the 'vesselID' column in train and get the integer IDs and the mapping
    vesselID, vesselID_mapping = pd.factorize(train['vesselId'])

    # Replace 'vessel_ID' column in train with integer IDs
    train['vesselId'] = vesselID

    # Create a dictionary from the mapping to apply the same to test
    vessel_to_ID = {vessel: idx for idx, vessel in enumerate(vesselID_mapping)}

    # Replace 'vesselID' in test using the same mapping from train
    test['vesselId'] = test['vesselId'].map(vessel_to_ID)
    
    # Replace 'vesselID' in schedules using the same mapping from train
    schedules['vesselId'] = schedules['vesselId'].map(vessel_to_ID)

    # Replace 'vesselID' in vessels using the same mapping from train
    vessels['vesselId'] = vessels['vesselId'].map(vessel_to_ID)

    # Replace 'portId' column with integer IDs
    #train['portId'] = pd.factorize(train['portId'])[0]

    # Factorize the 'portID' column in train and get the integer IDs and the mapping
    portID, portID_mapping = pd.factorize(train['portId'])

    # Replace 'port_ID' column in train with integer IDs
    train['portId'] = portID

    # Create a dictionary from the mapping to apply the same to test
    port_to_ID = {vessel: idx for idx, vessel in enumerate(portID_mapping)}

    # Replace 'portID' in test using the same mapping from train
    ports['portId'] = ports['portId'].map(port_to_ID)

    # Remove sog outliers
    #train = train[train['sog'] <= 40]

    return train, test, schedules, ports, vessels

In [46]:
ports.head()

,portId,name,portLocation,longitude,latitude,UN_LOCODE,countryName,ISO
0,61d36ed80a1807568ff9a064,Port of Algiers,Algiers,3.067222,36.773611,DZALG,Algeria,DZ
1,61d36ed80a1807568ff9a065,Port of Annaba,Annaba,7.772500,36.900556,DZAAE,Algeria,DZ
2,61d36edf0a1807568ff9a070,Port of Oran,Oran,-0.639722,35.712222,DZORN,Algeria,DZ
3,61d36ee00a1807568ff9a072,Port of Skikda,Skikda,6.905833,36.887500,DZSKI,Algeria,DZ
4,61d36ee10a1807568ff9a074,Port of Pago-Pago,Pago-Pago,-170.690556,-14.274167,ASPPG,American Samoa,AS


In [47]:
schedules.head()

,vesselId,shippingLineId,shippingLineName,arrivalDate,sailingDate,portName,portId,portLatitude,portLongitude
0,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-02 00:00:00+00:00,2023-10-03 00:00:00+00:00,Port of Brunswick,61d38499b7b7526e1adf3d54,31.140556,-81.496667
1,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-27 00:00:00+00:00,2023-10-27 00:00:00+00:00,Port of Southampton,61d3832bb7b7526e1adf3b63,50.902500,-1.428889
2,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-19 00:00:00+00:00,2023-10-20 00:00:00+00:00,Port of Bremerhaven,61d375e793c6feb83e5eb3e2,53.563611,8.554722
3,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-09 00:00:00+00:00,2023-10-10 00:00:00+00:00,Port of New York,61d38481b7b7526e1adf3d23,40.688333,-74.028611
4,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-09-25 00:00:00+00:00,2023-09-26 00:00:00+00:00,Manzanillo International Terminal,61d37d0199db2ccf7339eee1,9.372370,-79.879790


In [48]:
vessels.head()

,shippingLineId,vesselId,CEU,DWT,GT,NT,vesselType,breadth,depth,draft,enginePower,freshWater,fuel,homePort,length,maxHeight,maxSpeed,maxWidth,rampCapacity,yearBuilt
0,61a8e672f9cba188601e84ab,61e9f38eb937134a3c4bfd8b,6500,21200.0,58684,17606.0,83.0,32.0,22.20,NaN,0.0,NaN,NaN,OSLO,199.00,5.0,18.6,15.2,150.0,2000
1,61ec94f1a8cafc0e93f0e92a,61e9f38eb937134a3c4bfd8d,4902,12325.0,46800,NaN,83.0,31.0,NaN,NaN,14220.0,NaN,NaN,MONROVIA,182.00,NaN,NaN,NaN,NaN,2006
2,61e213d5d612676a0f0fb755,61e9f38eb937134a3c4bfd8f,5000,13059.0,46800,NaN,83.0,31.0,NaN,NaN,14220.0,NaN,NaN,SAINT JOHN'S,182.00,NaN,NaN,NaN,NaN,2010
3,61be24574ea00ae59d0fe388,61e9f38eb937134a3c4bfd91,4200,12588.0,39362,NaN,83.0,28.0,NaN,NaN,11060.0,NaN,NaN,NaN,167.00,NaN,NaN,NaN,NaN,2011
4,61a8e673f9cba188601e84ae,61e9f390b937134a3c4bfd93,7450,21052.0,75528,24391.0,83.0,37.2,22.23,NaN,13140.0,491.47,3236.78,Panama,199.98,NaN,NaN,NaN,NaN,2018


In [4]:
def feature_engineering(df_train,df_test,df_schedules,df_ports,df_vessels):  
    
    train = df_train.copy()
    test = df_test.copy()
    schedules = df_schedules.copy()
    ports = df_ports.copy()
    vessels = df_vessels.copy()
    features = pd.DataFrame()

    # Add the columns vesselId, time, latitude and longitude to the features from train
    features['vesselId'] = train['vesselId']
    features['time'] = train['time']
    features['latitude'] = train['latitude']
    features['longitude'] = train['longitude']

    # Sort by vesselID then time
    features = features.sort_values(['vesselId','time'])

    # Add lag features for every row in train
    features['latitude_lag1'] = train.groupby('vesselId')['latitude'].shift(1)    
    features['longitude_lag1'] = train.groupby('vesselId')['longitude'].shift(1)
    features['latitude_lag2'] = train.groupby('vesselId')['latitude'].shift(2)    
    features['longitude_lag2'] = train.groupby('vesselId')['longitude'].shift(2)
    features['latitude_lag3'] = train.groupby('vesselId')['latitude'].shift(3)    
    features['longitude_lag3'] = train.groupby('vesselId')['longitude'].shift(3)    

    # Remove the first row for every vesselId
    features = features.dropna()

    # New feature for if the vessel is moored or not
    features['not_under_way'] = train['navstat'].apply(lambda x: 1 if x == 5 or x == 1 else 0)
    features['under_way'] = train['navstat'].apply(lambda x: 1 if x == 0 or x == 8 else 0)

    # Add the column cog, sog, and rot to the features from train
    features['cog'] = train['cog']
    features['sog'] = train['sog']
    features['rot'] = train['rot']

    # Extract calendar features for 'etaRaw'
    features[['etaMonth', 'etaDay', 'etaHour', 'etaMinute']] = train['etaRaw'].str.extract(r'(\d{2})-(\d{2}) (\d{2}):(\d{2})')
    # Convert objects to integers
    features[['etaMonth', 'etaDay', 'etaHour', 'etaMinute']] = features[['etaMonth', 'etaDay', 'etaHour', 'etaMinute']].astype(int)

    # Split the time column into month, day, hour, minute and second columns
    features['month'] = train['time'].dt.month
    features['day'] = train['time'].dt.day
    features['hour'] = train['time'].dt.hour
    features['minute'] = train['time'].dt.minute
    features['second'] = train['time'].dt.second

    features.drop(columns=['time'], inplace=True)

    return features, test

In [40]:
features,test = preprocess(X_train,X_test,schedules,ports,vessels)
features,test = feature_engineering(features,test,schedules,ports,vessels)

In [9]:
features.head()

,vesselId,latitude,longitude,latitude_lag1,longitude_lag1,latitude_lag2,longitude_lag2,latitude_lag3,longitude_lag3,not_under_way,...,rot,etaMonth,etaDay,etaHour,etaMinute,month,day,hour,minute,second
3093,0,-35.16805,-56.53190,-35.16863,-56.63185,-35.16787,-56.77210,-34.74370,-57.85130,0,...,0,1,9,23,0,1,1,6,58,55
3140,0,-35.16715,-56.45306,-35.16805,-56.53190,-35.16863,-56.63185,-35.16787,-56.77210,0,...,0,1,9,23,0,1,1,7,15,56
3280,0,-35.16646,-56.40306,-35.16715,-56.45306,-35.16805,-56.53190,-35.16863,-56.63185,0,...,0,1,9,23,0,1,1,7,28,15
3586,0,-35.16544,-56.23866,-35.16646,-56.40306,-35.16715,-56.45306,-35.16805,-56.53190,0,...,0,1,9,23,0,1,1,8,3,35
4816,0,-35.08926,-55.50466,-35.16544,-56.23866,-35.16646,-56.40306,-35.16715,-56.45306,0,...,0,1,9,23,0,1,1,10,49,39


## Modelling

In [183]:
# Define features and targets
y = features[['latitude', 'longitude']]
x = features.drop(columns=['latitude', 'longitude'])

In [184]:
# Create a random forest regressor
#rf_model = RandomForestRegressor()

# Train the model
#rf_model.fit(x,y)

In [185]:
xgb_model = xgb.XGBRegressor()
xgb_model.fit(x,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## Predictions

In [41]:
# Find the last observed values for each vessel
def last_observed(df):
    last_obs = df.groupby('vesselId').last().reset_index()
    return last_obs

In [42]:
def prepare_test_for_predictions(test, features):
    test = test.copy()
    features = features.copy()

    # Find the last observed values for each vessel
    last_obs = last_observed(features)
    last_obs = last_obs.drop(columns=['longitude_lag3', 'latitude_lag3', 'month', 'day', 'hour', 'minute', 'second']).copy() 
    
    test = pd.merge(test, last_obs, on='vesselId', how='left')

    # Rename the columns latitude and longitude to last_latitude and last_longitude
    test.rename(columns={'longitude_lag2': 'longitude_lag3', 'latitude_lag2': 'latitude_lag3'}, inplace=True)
    test.rename(columns={'longitude_lag1': 'longitude_lag2', 'latitude_lag1': 'latitude_lag2'}, inplace=True)
    test.rename(columns={'longitude': 'longitude_lag1', 'latitude': 'latitude_lag1'}, inplace=True)

    # Fix the time column
    test['month'] = test['time'].dt.month
    test['day'] = test['time'].dt.day
    test['hour'] = test['time'].dt.hour
    test['minute'] = test['time'].dt.minute
    test['second'] = test['time'].dt.second
    test.drop('time', axis=1, inplace=True)

    test.drop('scaling_factor', axis=1, inplace=True)
    test.drop('ID', axis=1, inplace=True)

    return test

In [43]:
test = prepare_test_for_predictions(test,features)

In [44]:
features_412 = features[features['vesselId'] == 412]
features_412.tail()

,vesselId,latitude,longitude,latitude_lag1,longitude_lag1,latitude_lag2,longitude_lag2,latitude_lag3,longitude_lag3,not_under_way,...,rot,etaMonth,etaDay,etaHour,etaMinute,month,day,hour,minute,second
1514635,412,31.14645,-81.49792,31.14645,-81.49792,31.14645,-81.49791,31.14645,-81.49791,1,...,0,5,6,10,45,5,7,10,54,10
1514731,412,31.14645,-81.49792,31.14645,-81.49792,31.14645,-81.49792,31.14645,-81.49791,1,...,0,5,6,10,45,5,7,11,0,12
1521512,412,31.14648,-81.49789,31.14645,-81.49792,31.14645,-81.49792,31.14645,-81.49792,1,...,0,5,6,10,45,5,7,23,15,16
1521691,412,31.14648,-81.49789,31.14648,-81.49789,31.14645,-81.49792,31.14645,-81.49792,1,...,0,5,6,10,45,5,7,23,27,20
1521883,412,31.14647,-81.49789,31.14648,-81.49789,31.14648,-81.49789,31.14645,-81.49792,1,...,0,5,6,10,45,5,7,23,48,16


In [45]:
test_412 = test[test['vesselId'] == 412]
test_412.head()

,vesselId,latitude_lag1,longitude_lag1,latitude_lag2,longitude_lag2,latitude_lag3,longitude_lag3,not_under_way,under_way,cog,...,rot,etaMonth,etaDay,etaHour,etaMinute,month,day,hour,minute,second
0,412,31.14647,-81.49789,31.14648,-81.49789,31.14648,-81.49789,1,0,179.6,...,0,5,6,10,45,5,8,0,3,16
143,412,31.14647,-81.49789,31.14648,-81.49789,31.14648,-81.49789,1,0,179.6,...,0,5,6,10,45,5,8,0,36,14
282,412,31.14647,-81.49789,31.14648,-81.49789,31.14648,-81.49789,1,0,179.6,...,0,5,6,10,45,5,8,0,51,12
426,412,31.14647,-81.49789,31.14648,-81.49789,31.14648,-81.49789,1,0,179.6,...,0,5,6,10,45,5,8,1,12,14
551,412,31.14647,-81.49789,31.14648,-81.49789,31.14648,-81.49789,1,0,179.6,...,0,5,6,10,45,5,8,1,30,14


In [16]:
test.head()

,vesselId,latitude_lag1,longitude_lag1,latitude_lag2,longitude_lag2,latitude_lag3,longitude_lag3,not_under_way,under_way,cog,...,rot,etaMonth,etaDay,etaHour,etaMinute,month,day,hour,minute,second
0,412,31.14647,-81.49789,31.14648,-81.49789,31.14645,-81.49792,1,0,179.6,...,0,5,6,10,45,5,8,0,3,16
1,373,14.81694,120.29625,14.81688,120.29630,14.81691,120.29634,1,0,24.7,...,0,5,1,23,0,5,8,0,6,17
2,181,38.27895,10.78280,36.81120,10.29855,36.81119,10.29857,0,1,8.0,...,0,5,8,12,45,5,8,0,10,2
3,8,-43.53785,172.83522,-43.53800,172.83608,-43.53807,172.83608,1,0,321.3,...,0,5,7,1,15,5,8,0,10,34
4,65,48.53320,-6.12003,48.53133,-6.10695,48.55975,-6.07460,0,0,291.0,...,0,5,9,4,0,5,8,0,12,27


In [189]:
# Predict using the Random Forest model
predictions = xgb_model.predict(test)

In [190]:
# Create a DataFrame with the required format
predictions_df = pd.DataFrame(predictions, columns=['latitude_predicted', 'longitude_predicted'])
predictions_df['ID'] = range(len(predictions_df))
predictions_df = predictions_df[['ID', 'longitude_predicted', 'latitude_predicted']]

# Save the predictions to a CSV file
predictions_df.to_csv('predictions_5.csv', index=False, columns=['ID', 'longitude_predicted', 'latitude_predicted'])